[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/terrier-org/pyterrier/blob/master/examples/notebooks/schematics-demo.ipynb)

In [1]:
!pip install python-terrier pyterrier-t5 pyterrier-alpha pyterrier-doc2query

In [2]:
import pyterrier as pt
import pyterrier_alpha as pta
import pyterrier_doc2query
import pyterrier_t5

In [3]:
# A simple BM25 retriever, visualized
terrier_index = pt.Artifact.from_hf("macavaney/msmarco-passage.terrier")
terrier_index.bm25()

Java started (triggered by TerrierIndex.index_ref) and loaded: pyterrier.java, pyterrier.java.24, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]


08:26:25.259 [main] WARN org.terrier.structures.BaseCompressingMetaIndex -- Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.


TerrierRetr(/mnt/resources/pyterrier-cache/artifacts/7ead118630437940852142386f67ab62123a6ce372bb4b8cf12b06a76c8ccc25/data.properties,{'terrierql': 'on', 'parsecontrols': 'on', 'parseql': 'on', 'applypipeline': 'on', 'localmatching': 'on', 'filters': 'on', 'decorate': 'on', 'wmodel': 'BM25', 'bm25.k_1': 1.2, 'bm25.b': 0.75, 'decorate_batch': 'on', 'end': '999'},{'querying.processes': 'terrierql:TerrierQLParser,parsecontrols:TerrierQLToControls,parseql:TerrierQLToMatchingQueryTerms,matchopql:MatchingOpQLParser,applypipeline:ApplyTermPipeline,context_wmodel:org.terrier.python.WmodelFromContextProcess,localmatching:LocalManager$ApplyLocalMatching,qe:QueryExpansion,labels:org.terrier.learning.LabelDecorator,filters:LocalManager$PostFilterProcess,decorate:SimpleDecorateProcess', 'querying.postfilters': 'decorate:SimpleDecorate,site:SiteFilter,scope:Scope', 'querying.default.controls': 'wmodel:DPH,parsecontrols:on,parseql:on,applypipeline:on,terrierql:on,localmatching:on,filters:on,decorate:on', 'querying.allowed.controls': 'scope,qe,qemodel,start,end,site,scope,applypipeline', 'termpipelines': 'Stopwords,PorterStemmer'})

Hint: you can hover over the schematics for more information!

In [4]:
# A pipeline that performs RM3 query expansion
terrier_index.bm25() >> pt.terrier.rewrite.RM3(terrier_index.index_ref()) >> terrier_index.bm25()

08:26:25.929 [main] WARN org.terrier.structures.BaseCompressingMetaIndex -- Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.


(TerrierRetr(BM25) >> QueryExpansion(/mnt/resources/pyterrier-cache/artifacts/7ead118630437940852142386f67ab62123a6ce372bb4b8cf12b06a76c8ccc25,3,10,<org.terrier.querying.RM3 at 0x7f83f40a7dd0 jclass=org/terrier/querying/RM3 jself=<LocalRef obj=0xecd8130 at 0x7f83f417b990>>) >> TerrierRetr(BM25))

In [5]:
# Rank fusion between BM25 and BM25 with RM3
pta.RRFusion(
    terrier_index.bm25(),
    terrier_index.bm25() >> pt.terrier.rewrite.RM3(terrier_index.index_ref()) >> terrier_index.bm25()
)

08:26:26.556 [main] WARN org.terrier.structures.BaseCompressingMetaIndex -- Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.


In [6]:
# Let's add a re-ranker to the mix. It needs to load the text first.
monot5 = pyterrier_t5.MonoT5ReRanker()
pta.RRFusion(
    terrier_index.bm25(),
    terrier_index.bm25() >> pt.terrier.rewrite.RM3(terrier_index.index_ref()) >> terrier_index.bm25()
) >> terrier_index.text_loader() >> monot5

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


08:26:28.751 [main] WARN org.terrier.structures.BaseCompressingMetaIndex -- Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.


(<pyterrier_alpha.fusion.RRFusion object at 0x7f83f40a33d0> >> <pyterrier.terrier._text_loader.TerrierTextLoader object at 0x7f83f41eabf0> >> MonoT5(castorini/monot5-base-msmarco))

In [7]:
# Indexers are pipelines, too.
indexer = pt.terrier.TerrierIndex("./my-index").indexer()
indexer

text,str,Document text
docno,str,(External Document ID) String ID of document in collection


In [8]:
# You can chain pipelines here too. Here's Doc2Query
d2q = pyterrier_doc2query.Doc2Query(append=True, fast_tokenizer=True)
d2q >> indexer

(<pyterrier_doc2query.Doc2Query object at 0x7f82ea8edfc0> >> <pyterrier.terrier.index._IterDictIndexer_fifo object at 0x7f82ea8ee620>)